#### Warm-up exercise to recall iterations

In [ ]:
names = ["NGC 5128", "TXS 0506+056", "NGC 1068", "GB6 J1040+0617", "TXS 2226-184"]
distances = [3.7, 1.75e3, 14.4, 1.51e4, 107.1]  # Mpc
luminosities = [1e40, 3e46, 4.9e38, 6.2e45, 5.5e41] # erg/s

gal_cat = list(zip(names, distances, luminosities))

for name, dist, lum in gal_cat:
    print(f"{name:15s} D={dist:.2e} Mpc, L={lum:.2e} erg/s")

# (Almost) all about functions

Functions make code more maintainable and more organized/modular. If there is a task that you repeat several times in your program, packaging it as a function is much better practice than copying the code snippet for the task repeatedly. If you update the task, you only need to update the function once. Functions allow you to clearly separate different sub-tasks in your program, rather than writing one long list of commands with comments to indicate the sub-tasks.

In general, a function takes ***arguments*** as input, processes them (via ***statements*** in the function body), and ***returns*** a result as output. A generic function looks like this:

In [ ]:
def <function_name>([<parameters>]):
    <statement(s)>

The function is executed only if it is called in the main body of the code. Here's a real example:

In [ ]:
from math import pi

def calc_flux(luminosity, distance_mpc):
    distance_cm = distance_mpc * 3e24
    flux = luminosity / (4 * pi * distance_cm ** 2)
    return flux

lum, dist = 4e45, 100

flux = calc_flux(lum, dist)

print(f"{flux:.2e} erg/s")

Here, two arguments are passed to the function. Since there are no default values for the arguments defined, the number of arguments passed must match the number of parameters the function expects, otherwise an error will be thrown. Only the ordering shows which argument is mapped to which parameter; these are called positional arguments.

## Indentation
You'll notice above that indentation is important - it delimits the body of the function, and separates between the main code and the function definition. 
- *indentation* means shifting a line of code by either a given number of spaces or a tab (`Tab` key);
- a tab is a *single* special character that is visualised as an empty space;
- tab-style indentation may have been popular in the past, but today the standard is space-style indentation using 4 whitespaces;
- most editors will produce 4 whitespaces by default (or can be set up to do so!)

### In python
- indentation in python is ***part of the syntax!***
- indentation delimits the code of a function, an `if/elif/else` clause, a loop etc.
- any number of spaces is recognised, but it has to be consistent

If you forget a `return` statement, your function will return `None`

In [ ]:
def add_one(num):
    num += 1
    #return num
    
print(add_one(10))

## Scope

In [ ]:
count = 0

def bad_function():
    count += 1
    return count

# bad_function() # Calling this function will return an error

Variables defined inside the namespace of a function are *local* to that function. The namespace refers to the defined names and objects that the names refer to.

Variables created outside a function are called *global* variables

In [ ]:
def add_one(count):
    count += 1 # count is a local variable because it's an argument
    return count


count = 1      # global 
print(count)

add_one(count) # this is returning 2 and we're doing nothing with the result
print(count)   # global variable is unaffected

count = add_one(count) # only now are we updating the variable count 
print(count)

Best practice: call differently your arguments, local variables and global variables

In [ ]:
# Exercise: rewrite the code snippet above so that the local and global variables are clearly defined
def add_one(n):
    res = n + 1  # Both n and res are local variables
    return res

count = 1      # Calling our global and local variables differently
               # avoids confusion
print(count)
count = add_one(count) # this returns 2 and we're replacing our global variable with it
print(count)   # global variable is changed

Note that scope is important not just with functions, but with classes, which we'll see next week, and comprehension, which we saw last week. For example this code snippet returns an error:

In [ ]:
[item for item in range(5)]
item

However this is not an issue with a standard `for` loop, though it might not give you the behavior you want.

In [ ]:
for item in range(5):
    print(item)

item

### Function returns

A function can return *any* type, including lists, dictionaries, booleans, or even functions

In [ ]:
def is_detectable(flux):
    return flux > 1e-11

print(is_detectable(1e-12))

In [ ]:
# Exercise: use function is_detectable and galaxy catalog to print whether each galaxy in catalog is detectable or not
def is_detectable(luminosity, distance):
    flux = calc_flux(luminosity, distance)
    return flux > 1e-11

for name,dis,lum in gal_cat:
    if is_detectable(lum, dis):
        print(f"{name:15s} is detectable")
    else:
        print(f"{name:15s} is not detectable")

A function terminates the first time that `return` is called - beware of pitfalls!

In [ ]:
def find_first_detectable(catalog):
    for name, dis, lum in catalog:
        if is_detectable(lum, dis):
            return name

firstname = find_first_detectable(gal_cat)
print(f"First resolved galaxy: {firstname}")  # I can do this
#print(f"First resolved galaxy: {firstname:s}") # But I wouldn't be able to do this if the flux limit is too high, e.g. 1e-10

The problem with the above function is that if there are no elements that satisfy our requirement, the `return` statement will never be called and the function will return a `NoneType`. Let's fix that:

In [ ]:
# Exercise: rewrite the above function to always return a string

def find_first_detectable(catalog):
    firstname = "None!"
    for name, dis, lum in catalog:
        if is_detectable(lum, dis):
            firstname = name
    return firstname

firstname = find_first_detectable(gal_cat)
print(f"First resolved galaxy: {firstname:s}") # Now I know that a string will always be returned

Python functions are extremely flexible and can even return multiple variables of different types

In [ ]:
def assess_flux(luminosity, distance):
    flux = calc_flux(luminosity, distance)
    isdetect = is_detectable(luminosity, distance)
    return flux, isdetect

results  = assess_flux(1e45, 100) # above detectability threshold
# results  = assess_flux(1e43, 100) # below detectability threshold
print(results)

if results[1]:
    print(f"A flux of {results[0]:.2e} erg/cm2/s is detectable!\n")

# A better syntax is to "unpack" the result into different variables:

flx, isdet = assess_flux(1e45, 100) # above detectability threshold
# flx, isdet = assess_flux(1e43, 100) # below detectability threshold

print(flx, isdet)

if isdet:
    print(f"A flux of {flx:.2e} erg/cm2/s is detectable!\n")

### Keyword arguments

In [ ]:
from math import sqrt

def quadratic(a, b, c):
    x1 = -b / (2*a)
    x2 = sqrt(b**2 - 4*a*c) / (2*a)
    return (x1 + x2), (x1 - x2)

print(quadratic(a=31, b=93, c=62))
print(quadratic(c=62, a=31, b=93))

But positional arguments must come first, if we use a mix of both.

In [ ]:
# This will work
a, b = 31, 93
print(quadratic(a, b, c=62))

In [ ]:
# This will not
a, c = 31, 62
print(quadratic(a, b=93, c))

### Default parameters

We can give some parameters default values. 

In [ ]:
def is_detectable(luminosity, distance, threshold=1e-11): # luminosity and distance are positional: 
                                                          # they must always be passed then calling 
                                                          # the function. threshold is keyword, and will
                                                          # be defaulted to 1e-11 if I don't pass it
                                                          # to the function
                                                        
    flux = calc_flux(luminosity, distance)
    return flux > threshold

print(is_detectable(1e45,100)) # I don't give any value of threshold,
                        # so Python assumes the default value 
                        # I defined in the function (in this case 1e-11) 
        
print(is_detectable(1e45,100, 1e-12)) # Now Python takes the value I passed to the function

print(is_detectable(1e45,100, 1e-9))
        

These defaulted parameters must come ***after*** all the undefined arguments

In [ ]:
# Trying to define a function like this will throw an error:

def is_detectable(luminosity, threshold=1e-11, distance):
    flux = calc_flux(luminosity, distance)
    return flux > threshold

When you add a parameter to a function, always remember to update all the functions that depend on it! 

In [ ]:
def find_first_detectable(catalog, threshold=1e-11):
    firstname = "None!"
    for name, dis, lum in catalog:
        if is_detectable(lum, dis, threshold): # I pass on the threshold
                                               # parameter to all functions
                                               # that depend on it
            firstname = name
    return firstname

firstname = find_first_detectable(gal_cat)
print(firstname)

### Variable length argument lists
In the examples above, we call a function that takes one luminosity and one distance. What if we want to pass in e.g. a group of distances?

In [ ]:
def calc_dist_cm(*args):
    for i in args:
        distance_cm = i * 3e24
        print(f"Distance: {distance_cm} cm")

calc_dist_cm(3.7, 1750.0, 14.4, 15100.0, 107.1)

More useful is passing a tuple packed up from e.g. a list:

In [ ]:
def calc_dist_cm(*args):
    for i in args:
        distance_cm = i * 3e24
        print(f"Distance: {distance_cm} cm")

calc_dist_cm(*distances)

We can also use a similar syntax for dictionaries.

In [ ]:
galaxy_luminosities = dict(zip(names, luminosities))

def print_galaxies(**kwargs):
    for k, v in kwargs.items():
        print(f"Name: {k}, Luminosity {v} erg/s ")

print_galaxies(**galaxy_luminosities)

## Recursion

Functions can not only depend on other functions, but also on themselves. 

In [ ]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
    
for i in range(12):
    print(fibonacci(i))

In [ ]:
# Exercise: write a function for calculating a factorial, and print 0! through 9!
def factorial(n):
    if n <= 1:
        return 1
    else:
        return n * factorial(n-1)

for i in range(10):
    print(i, factorial(i))

## Type hints
- python is dynamically typed: you can do whatever you want and there will be little control about the types you use!
- from python 3.5 *type hints* are supported: we can indicate what types a function is supposed to take as arguments and what type it returns!

In [ ]:
def f(a : int, b : int) -> str:
    if a > b:
        return f"{a} is greater than {b}"
    else:
        return f"{a} is less than or equal to {b}"

print(f(1, 2))

print(f(1.5, 3.5))

- the python interpreter does not complain if you don't respect type hints, after all it is a *dynamically typed* language!
- type hints are useful **to you** to remember how a function is supposed to behave: they may seem (and probably are) unnecessary at this level but it is important to **pick good habits** from the start! 
- there are tools known as **static type checkers** (one is `mypy`) that can check if your code respect all the type declarations.

### Defining functions in one line with `lambda`
`lambda` is an example of python's support for functional programming. They provide a compact alternative for simple functions. They consist of a keyword (`lambda`) a variable, and a body.

In [ ]:
# The above factorial function can be rewritten as

factorial = lambda n: n * factorial(n-1) if n > 1 else 1

print(factorial(4.), factorial(10))

`lambda` functions can also take several arguments, but should be used only for simple tasks so as not to become unreadable. Notice that the arguments are separated by commas, but not enclosed in parentheses. 

In [ ]:
hypothenuse = lambda x,y: (x ** 2 + y ** 2) ** 0.5

sa, sb = 3, 4
sc = hypothenuse(sa,sb)
print(f"A={sa}, B={sb} -> C = {sc}")

You can also pass in values for the arguments in the same line.

In [ ]:
hypothenuse = (lambda x,y: (x ** 2 + y ** 2) ** 0.5)(3,4)
print(hypothenuse)

`lambda` functions have some key differences from standard functions. They don't support statements within the body of the function, or type hints. For example, neither of the examples below will work.

In [ ]:
(lambda x: return x**2)(2)

This is the correct syntax.

In [ ]:
(lambda x: x**2)(2)

In [ ]:
hypothenuse = (lambda x: int,y: int: (x ** 2 + y ** 2) ** 0.5)(3,4)

In contrast to functions, `lambda` functions are invoked immediately, which can be particularly convenient within a Jupyter notebook.

### Docstrings

Adding docstrings to your functions will help you become a better programmer and a better scientist!

Your future self and your collaborators will appreciate it, becuase your code will be clearer, which will help prevent errors in your results. (On top of this, your future employers will love seeing your neatly documented code on your GitHub repository) 

Even in the most obvious cases, your docstring should be at least one line: 

In [ ]:
def add_one(n):
    """Calculate n+1 and return the result."""
    res = n + 1  
    return res

def check_script():
    """Check if the script is running."""

If the funtion does something more complex, you should write a more complete docstring, in this general form:

In [ ]:
def my_function(par1, par2):
    """
    One-line description of the purpose of the function.

    If necessary, you can add here a second paragraph explaining in detail
    the rationale and usage of the function, including an example if 
    necessary. By using three quotation marks, every line in between is 
    interpreted as part of the same string. So use line breaks like this 
    to keep your lines short.
    
    Args:
        par1: a number
        par2: a second number
    
    Returns:
        The result of some operation on our input
    """
    res = some_operation(par1, par2)
    return res

Strings written in this fashion will become the docstring of the function, which will help your future self or your collaborators understand your code:

In [ ]:
help(my_function)

In [ ]:
help(len)

In the Jupyter environment, you can also get the docstring by pressing Shift+Tab on a function.

Here I'm using my docstrings in the so-called Google format. There are other options, as you can read about online:

https://stackoverflow.com/questions/3898572/what-are-the-most-common-python-docstring-formats https://betterprogramming.pub/3-different-docstring-formats-for-python-d27be81e0d68

For more on how to write good docstrings, check out the PEP conventions:

https://peps.python.org/pep-0257/